Building a tool that takes a technical question, and responds with an explanation from each model.

In [29]:
import ollama
from openai import OpenAI
import os
from IPython.display import Markdown, display, update_display

api_key_gpt = os.getenv('OPENAI_API_KEY') #OPENAI_API_KEY is in Path variable on local machine
if api_key_gpt and api_key_gpt.startswith('sk-proj-') and len(api_key_gpt)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL_GPT = 'gpt-4o-mini'
openai = OpenAI()

OLLAMA_API = "http://localhost:11434/api/chat" #url on local box
MODEL_LLAMA = 'llama3.2'

API key looks good so far


In [2]:
system_prompt = "You are a technical expert.  You will be asked a question \
    either about some code or some other computer or software related question. \
    Please analyze and provide a succinct analysis or explanation , in as simple of terms of possible, of what the \
    user is asking."

In [3]:
#Just replace this text to ask something new
user_question = """ 
    Can you explain what purpose 'yield' serves in the python code:
    yield from {book.get("author") for book in books if book.get("author")}
    """

In [ ]:
#Gpt 4o-mini answer w/streaming
stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_question}
          ],
        stream=True #This allows data to 'flow' back in chunks, rather than a single text of response
    )
#Initialize empty response
response = ""

display_handle = display(Markdown(""), display_id=True) #empty markdown element
for chunk in stream: # as data arrives
    response += chunk.choices[0].delta.content or '' #extract text from response chunk, where delta contains only the newest part of the response
    response = response.replace("```","").replace("markdown", "") #Ensure we don't add None but rather an empty string
    update_display(Markdown(response), display_id=display_handle.display_id)

In Python, `yield` is used in functions to make them generators. A generator is a type of iterable that allows you to iterate over a sequence of values without storing the entire sequence in memory at once.

In the code you provided:

python
yield from {book.get("author") for book in books if book.get("author")}


Here's what each part means:

1. **`yield from`**: This keyword is used to yield all values from an iterable one at a time. It allows the function to delegate part of its operations to another iterable (in this case, a set comprehension).

2. **`{book.get("author") for book in books if book.get("author")}`**: This is a set comprehension that creates a set of authors from a list of `books`. It goes through each `book`, retrieves the author using `book.get("author")`, and includes this author in the set only if it's not `None`.

Summarizing, the entire line will produce each distinct author of the books in the `books` list, one at a time, when iterating through the generator. This allows for memory-efficient handling of potentially large datasets while also ensuring that you only work with unique authors.

In [13]:
#llama3.2 to answer
response = ollama.chat(
        model = MODEL_LLAMA,
        messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ]
    )
print(display(Markdown(response['message']['content'])))

**Explanation of `yield`**

In Python 3, the `yield` keyword is used to define generators. A generator is a special type of iterable that can be used to generate a sequence of values on-the-fly, rather than computing them all at once and storing them in memory.

When you use `yield`, you're essentially creating a function that produces a value and then pauses its execution, returning control back to the caller. The next time the function is called, it resumes from where it left off, yielding another value, and so on.

**In this specific code**

The line `yield from {book.get("author") for book in books if book.get("author")}` uses a feature of Python 3.5+ called "yield from" to delegate the generation of values to an existing iterator.

Here's what's happening:

1. `{book.get("author") for book in books if book.get("author")}` is a generator expression that iterates over the `books` list, filters out items with no author, and yields their authors.
2. `yield from ...` takes the result of the generator expression (an iterator) and "yields from" it, effectively delegating its generation of values to this new iterator.

In essence, this code is saying: "Take the filtered book authors and yield them one by one, without storing them all in memory at once."

**Benefits**

This approach has several advantages:

* Memory efficiency: We're not creating a large list or dictionary to store the authors; instead, we generate them on-the-fly.
* Flexibility: This code can be used with any iterable (like a database query result) that produces author values.

In summary, `yield` in this context allows us to lazily generate and yield book authors without storing them all in memory at once.

None
